In [1]:
import numpy as np
from scipy.optimize import minimize
from scipy.stats import norm
import math

Defining values

In [2]:
R=np.array([[1,0,0],       #the correlation matrix
            [0,1,0],
            [0,0,1]
            ])
L=np.linalg.cholesky(R)
SD=np.array([[2,0,0],       #standard deviations
            [0,2,0],
            [0,0,6]
            ])
M=np.array([[10],       #mean
            [3],
            [25]
            ])
##THE COEFFICIENT MATRIX
print(SD@L)

[[2. 0. 0.]
 [0. 2. 0.]
 [0. 0. 6.]]


AFORM Analysis for first finding the design point

In [3]:
def objective(A):
    return np.sqrt(np.dot(A,np.transpose(A)))

def constraint(A):
    return (A[0]*2+10)*(A[1]*2+3)-(A[2]*6+25)
    
Init_guess=np.zeros(3)
Init_guess=np.array([-1,2,-3])
#######################

b=(-100,100)
bnds=(b,b,b)
con={'type':"eq","fun":constraint}
sol=minimize(objective,Init_guess,method="SLSQP",bounds=bnds,constraints=con)
print(sol)
MPP = ( SD @ L @ sol['x'].reshape(3,1) + M ).flatten()
print("\nThe design MPP Point as per AFORM is: \n", MPP)  #THE MPP POINT

 message: Optimization terminated successfully
 success: True
  status: 0
     fun: 0.2325269730966812
       x: [-5.610e-02 -2.159e-01  6.553e-02]
     nit: 9
     jac: [-2.396e-01 -9.295e-01  2.804e-01]
    nfev: 41
    njev: 9

The design MPP Point as per AFORM is: 
 [ 9.88780454  2.56813014 25.39316919]


Importance sampling on monte carlo simulation for failure probability estimation

In [4]:
# GENERATING VALUES

n_sample = 10 # defining number of samples
sd=SD.flatten()
sd=sd[sd!=0]
m=M.flatten()

u1, u2, u3 = np.random.rand(n_sample), np.random.rand(n_sample), np.random.rand(n_sample)
x1, x2, x3 = norm.ppf(u1, loc=MPP[0], scale=sd[0]), norm.ppf(u2, loc=MPP[1], scale=sd[1]), norm.ppf(u3, loc=MPP[2], scale=sd[2])

G = x1 * x2 - x3                   # Limit state function of interest
I = [1 if y>=0 else 0 for y in G]

f1, f2, f3 = norm.pdf(x1, loc=m[0], scale=sd[0]), norm.pdf(x2, loc=m[1], scale=sd[1]), norm.pdf(x3, loc=m[2], scale=sd[2]) # PDFs
f= f1 * f2 * f3                    # because f1, f2 and f3 are statistically independent
s1, s2, s3 = norm.pdf(x1, loc=MPP[0], scale=sd[0]), norm.pdf(x2, loc=MPP[1], scale=sd[1]), norm.pdf(x3, loc=MPP[2], scale=sd[2])
s = s1 * s2 * s3                   # because s1, s2 and s3 are statistically independent
w = f/s                            # weight calculation for importance based sampling
final = w*I 
mean = np.mean(final)
COV = np.sqrt( (1- mean)/(n_sample*mean) )
print(" The failure probability (pf) is ",mean)
print("\n The COV is ",COV*100)



 The failure probability (pf) is  0.3274700260348623

 The COV is  45.31792803436899
